# Loss Function Experiments


## Setup


In [ ]:
%load_ext autoreload

import numpy as np
import math
import random
import os
import os.path
import torch
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

from tqdm.notebook import tqdm
from ipywidgets import interact 
import gc
import open3d as o3d

from src.elements import *
from src.ifc import *
from src.preparation import *
from src.visualisation import *
from src.chamfer import *
from src.utils import *

from src.morph import *

random.seed = 42


This notebook contains experiments on loss functions.

Specifically it contains;

1. Analysis of results from sphere morphing, including consistency metrics
2. Visualising point-wise distances from a loss function
3. Measuring loss correlation in results from completion models
4. Visualising loss curves for model training

#### data loading and pre-processing

All data from sphere morphing must be loaded for analysis.


In [ ]:
# helper code to recombine batched output from balancedCD
file_prefix = "sphere/balanced_metrics"
icd_chamfer_list, icd_emd_list, icd_assignment_list = [], [], []

for i in range(1,9):
    with open(file_prefix + str(i) + ".pkl", "rb") as f:
        chamfer, emd, ass = pickle.load(f)
        print(len(ass), len(ass[0]), len(ass[0][0]), ass[0][0][0].shape)
        icd_chamfer_list.append(chamfer)
        icd_emd_list.append(emd)
        icd_assignment_list.append(ass[0])

icd_assignment_list = np.array(icd_assignment_list)        
icd_assignment_list = np.transpose(icd_assignment_list, axes=(1,2,0,3,4))
icd_assignment_list = np.reshape(icd_assignment_list, (101, 2, 8*150, 4096))

In [ ]:
balanced_cd = np.sum(np.array(icd_chamfer_list), axis=0)
balanced_emd = np.sum(np.array(icd_emd_list), axis=0)
print(balanced_emd.shape)

In [ ]:
# load metrics
loss_funcs = ["emd"]
loss_funcs = ["emd",  "cyclic"]
chamfer_list, emd_list = [], []
for loss_func in loss_funcs:
    if loss_func == "balanced":
        continue
    with open("sphere/" + loss_func + "_metrics.pkl", "rb") as f:
        chamfer, emd, ass = pickle.load(f)
        chamfer_list.append(chamfer)
        emd_list.append(emd)


In [ ]:
if "balanced" in loss_funcs:
    chamfer_list.append(balanced_cd)
    emd_list.append(balanced_emd)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 5))
plt.rcParams.update({'font.size': 12})
plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=14) 
#loss_funcs = ["infoCD", "CD", "EMD", "UniformCD"]

plot_dists(axes[1], chamfer_list, loss_funcs, "Chamfer Distance", "iterations", log=True)
plot_dists(axes[0], emd_list, loss_funcs, "Earth Mover's Distance", "iterations", log=True)
#print("EMD infocd", emd_list[0][-1], "chamfer", emd_list[1][-1],  "emd", emd_list[2][-1], "BALANCED", emd_list[3][-1])
#print("CD infocd", chamfer_list[0][-1], "chamfer", chamfer_list[1][-1], "emd", chamfer_list[2][-1], "balanced", chamfer_list[3][-1])


In [ ]:
print(len(ass), len(ass[0]), len(ass[0][0]), len(ass[0][0][0]), ass[0][0][0][0].shape)
#print(len(ass), len(ass[0]), ass[0][0].shape, len(ass[0][0]), ass[0][0][0].shape)
print(emd_list[1][0])

In [ ]:
# load losses
# single, not batch
loss_types = ["reverse", "chamfer", "emd", "pair"]
losses = []

for loss_func in loss_types:
    with open("sphere/loss_" + loss_func + ".pkl", "rb") as f:
        losses.append(pickle.load(f))
        
plot_dists(losses, loss_types, "loss function comparison")


#### consistency metrics

We measure;

1. backward assignment consistency
2. coorrespondence coverage
3. correspondence stability


In [ ]:
# measure conssistency between forward and backward correspondences for chamfer distance
# optionally compare against the ideal assignment, as measured by EMD
# def measure_assignment_consistency(assignment, emd=None):
#     reverse_assignment = torch.gather(assignment[0], 0, assignment[1])
#     expected = torch.arange(assignment[0].shape[0], device=torch.device("cuda"))
#     consistency = torch.sum(torch.eq(expected, reverse_assignment).long())
#     print("consistency", consistency.item(), len(torch.unique(assignment[0])), len(torch.unique(assignment[1])))
    
#     if emd is not None:
#         #print(emd[:5], assignment[0][:5], assignment[1][:5])
#         emd_consistency = torch.sum(torch.eq(emd, assignment[0]).long())
#         #print("emd_consistency", emd_consistency.item(), len(torch.unique(emd)))


In [ ]:
# measure conssistency for EMD approx correspondences
# this is different from other distances as only a one way assignment is returned
def measure_batch_assignment_consistency_emd(assignment):
    consistencies = np.array([4096 for i in range(assignment.shape[-1])])
    cuda = torch.device("cuda")

    uniques = []
    changes = []
    for i in tqdm(range(len(assignment))):
        class_assignment = torch.tensor(assignment[i], device=cuda)
        
        unique = 0
        for j in range(len(class_assignment)):
            unique += len(torch.unique(class_assignment[j]))
        unique = (unique/len(class_assignment))
        uniques.append(unique)
        
        # check rate of change of matches
        if i==0:
            change = 0
        else:
            change = (assignment[i] == assignment[i-1]).sum()
            change = (change/len(class_assignment[0]))
        changes.append(change)
        
        print("un", unique, "change", change)
        
    # save results
    with open("sphere/" + "emd" + "_consistency.pkl", "wb") as f:
        pickle.dump([consistencies, uniques, changes], f)

In [ ]:
measure_batch_assignment_consistency_emd(ass)

In [ ]:
# measure consistency between forward and backward correspondences
# measure number of unique assignments
# measure rate of change of assignments
def measure_batch_assignment_consistency(assignment, loss):
    cuda = torch.device("cuda")
    # loop through iterations
    consistencies = []
    uniques = []
    changes = []
    for i in tqdm(range(len(assignment))):
        
        # check reverse consistency
        class_assignment = torch.tensor(assignment[i], device=cuda)
        reverse_assignment = torch.gather(class_assignment[0], 1, class_assignment[1])
        #print("r", reverse_assignment.shape)
        expected = torch.arange(class_assignment[0].shape[1], device=cuda)
        expected = expected.repeat(class_assignment[0].shape[0], 1)
        #print("e", expected.shape)
        consistency = torch.sum(torch.eq(expected, reverse_assignment).long())/len(class_assignment[0])
        consistencies.append(consistency.item())
        
        # check uniqueness of matches
        unique = 0
        for j in range(len(class_assignment[0])):
            unique += (len(torch.unique(class_assignment[0][j])) +
                       len(torch.unique(class_assignment[1][j])))
        unique = (unique/len(class_assignment[0]))/2
        uniques.append(unique)
        
        # check rate of change of matches
        if i==0:
            change = 0
        else:
            change = (assignment[i] == assignment[i-1]).sum()
            change = (change/len(class_assignment[0]))/2
        changes.append(change)
        
        print("un", unique, "consistency", consistency.item(), "change", change)

    
    # save results
    with open("sphere/" + loss + "_consistency.pkl", "wb") as f:
        pickle.dump([consistencies, uniques, changes], f)


In [ ]:
# preprocess
loss = "cyclic"
#ass = np.array(ass)

if loss == "infocd":
    ass = np.transpose(ass, axes=(1,2,0,3,4,5))
    ass = np.reshape(ass, (101, 2, 8*150, 4096))
    

if loss == "cyclic":
    ass = np.transpose(ass, axes=(1,2,0,3,4,))
    ass = np.reshape(ass, (101, 2, 8*150, 4096))
    
if loss == "chamfer":
    print(ass.shape)
    ass = np.transpose(ass, axes=(1,2,0,3,4))
    ass = np.reshape(ass, (101, 2, 8*150, 4096))
    print(ass.shape)

if loss == "emd":
    ass = np.array(ass)
    ass = np.transpose(ass, axes=(1,0,2,3))
    ass = np.reshape(ass, (101, 8*150, 4096))
print(ass.shape)

In [ ]:
#measure_batch_assignment_consistency(icd_assignment_list, loss="cyclic")
measure_batch_assignment_consistency(ass, loss=loss)


In [ ]:
# plot consistency metrics

loss_funcs = ["infocd", "chamfer", "emd", "UniformCD", "cyclic"]
unique_list, consistency_list, change_list = [], [], []
for loss_func in loss_funcs:
    with open("sphere/" + loss_func + "_consistency.pkl", "rb") as f:
        consistency, unique, change = pickle.load(f)
        consistency_list.append(consistency)
        unique_list.append(unique)
        change_list.append(change)
print(change_list[0][-1], change_list[1][-1])

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 5))
plt.rcParams.update({'font.size': 12})
plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=14) 

limit= 50
loss_funcs = ["InfoCD", "CD", "EMD", "UniformCD", "SCD"]

plot_dists(axes[0], consistency_list, loss_funcs, title="Backward consistency", 
           ylabel="no. of points", xlabel="Iterations", log=False, limit=limit, legend=False)
plot_dists(axes[1], unique_list, loss_funcs, title="Point coverage", 
           ylabel="", xlabel="Iterations", log=False, limit=limit, legend=False)
plot_dists(axes[2], change_list, loss_funcs, title="Correspondence variation", 
           ylabel="", xlabel="Iterations", log=False, limit=limit)


In [ ]:
# compare correspondences
#TODO: include consistency in top5 matches?
loss_func = "emd"
with open("sphere/assignments_" + "emd" + ".pkl", "rb") as f:
    emd_assignment = pickle.load(f)

with open("sphere/assignments_" + loss_func + ".pkl", "rb") as f:
    assignment = pickle.load(f)
    
for i, ass in enumerate(assignment):
    #measure_assignment_consistency(ass, emd_assignment[i][0])
    measure_assignment_consistency(ass)


#### Point-wise correspondence distance visualisation


In [ ]:
# load clouds
tgt_path = "sphere/scaled_lamp4_su.pcd"
l1_path = "sphere/scaled_lamp4_su2.pcd"
l2_path = "sphere/scaled_lamp4_sr3.pcd"
l3_path = "sphere/scaled_lamp4_sl2.pcd"

tgt = np.array(o3d.io.read_point_cloud(tgt_path).points)
l1 = np.array(o3d.io.read_point_cloud(l1_path).points)
l2 = np.array(o3d.io.read_point_cloud(l2_path).points)
l3 = np.array(o3d.io.read_point_cloud(l3_path).points)


In [ ]:
# measure point-wise distance between two clouds
def get_point_distance(src, tgt, loss="chamfer"):
    cuda = torch.device("cuda")
    src_tensor = torch.tensor([src], device=cuda)
    tgt_tensor = torch.tensor([tgt], device=cuda)
    chamferDist = ChamferDistance()

    if loss == "chamfer":
        nn = chamferDist(
            src_tensor, tgt_tensor, bidirectional=True, return_nn=True)
        weights = nn[1].dists[0,:,0]
        #print( torch.sum(nn[1].dists))
        print("w", weights.shape)
        loss = torch.mean(nn[1].dists)
        
    elif loss == "balanced":
        dist_0, dist_1 = calc_balanced_chamfer_loss_tensor(src_tensor, tgt_tensor, return_dists=True, k=32)
        loss = torch.mean(dist_1[0])
        #print( torch.sum(dist_1[0]))
        weights = dist_1[0]
    
    return weights.detach().cpu().numpy(), loss.item()

In [ ]:
# produce a colour map based on the weights of a point cloud
def visualise_point_loss(weights, high, low, colormap_name='plasma'):
    # normalise weights
    # weights = np.log(weights)
    # high = np.log(high)
    # low = np.log(low)
    diff = high - low
    weights = (weights - low) / diff
    
    # map colour
    colours = np.zeros((len(weights), 4))
    colormap = plt.get_cmap(colormap_name)
    for j, pt in enumerate(weights):
        colours[j] = colormap(pt)

    return colours[:,:3]

In [ ]:
%autoreload 2

def get_point_weights(tgt, l1, l2, l3, loss_func="chamfer"):
    w1, loss1 = get_point_distance(tgt, l1, loss_func)
    w2, loss2 = get_point_distance(tgt, l2, loss_func)
    w3, loss3 = get_point_distance(tgt, l3, loss_func)
    

    high1, low1 = np.max(w1), np.min(w1)
    high2, low2 = np.max(w2), np.min(w2)
    high3, low3 = np.max(w3), np.min(w3)

    high = max(high1, high2, high3)
    low = min(low1, low2, low3)
    
    losses = [loss1, loss2, loss3]
    print(loss_func, "loss1", loss1, "loss2", loss2, "loss3", loss3)
    
    return high, low, w1, w2, w3, losses



cd_high, cd_low, cd_w1, cd_w2, cd_w3, cd_losses = get_point_weights(tgt, l1, l2, l3, loss_func="chamfer")
balanced_high, balanced_low, balanced_w1, balanced_w2, balanced_w3, balanced_losses = get_point_weights(tgt, l1, l2, l3, loss_func="balanced")

high, low = max(cd_high, balanced_high), min(cd_low, balanced_low)

#colours = visualise_point_loss(balanced_w1, high, low, 'plasma_r')
#colours = visualise_point_loss(balanced_w4, balanced_high, balanced_low, 'plasma_r')
colours = visualise_point_loss(cd_w3, high, low, 'plasma_r')
print("colours", (colours!=0).sum(), colours.shape)
point_cloud = o3d.geometry.PointCloud()
#print(colours, cloud.shape)
point_cloud.points = o3d.utility.Vector3dVector(l3)
point_cloud.colors = o3d.utility.Vector3dVector(colours)
o3d.visualization.draw_geometries([point_cloud])

In [ ]:
point_cloud = o3d.geometry.PointCloud()
#print(colours, cloud.shape)
point_cloud.points = o3d.utility.Vector3dVector(tgt)
point_cloud.paint_uniform_color([0.1, 0.8, 0.6])

o3d.visualization.draw_geometries([point_cloud])

In [ ]:

print(normalize([balanced_losses]))
X = np.arange(3)

fig = plt.figure()

ax = fig.add_axes([0,0,1,1])
#ax.bar(X + 0.00, normalize([balanced_losses])[0], color = 'b', width = 0.25)
ax.bar(X + 0.25, normalize([cd_losses])[0], color = 'mediumslateblue', width = 0.8)
ax.set(ylabel="Loss (normalised)")
ax.set(title="Chamfer distance")

x1,x2,y1,y2 = plt.axis()  
plt.axis((x1,x2,0,0.7))
ax.set_xticks([])

In [ ]:
X = np.arange(3)
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])

plt.rcParams.update({'font.size': 22})
plt.rc('xtick', labelsize=10) 
#ax.bar(X + 0.00, normalize([balanced_losses])[0], color = 'b', width = 0.25)
ax.bar(X + 0.25, normalize([balanced_losses])[0], color = 'indianred', width = 0.8)
ax.set(ylabel="Loss (normalised)")
ax.set(title="UniformCD distance")
x1,x2,y1,y2 = plt.axis()  
plt.axis((x1,x2,0,0.7))
ax.set_xticks([])

#### CD EMD scatterplot


In [ ]:
# on PCN results
dataset = "mvp"

if dataset == "mvp":
    chamfer_path = "../experiments/Density_aware_Chamfer_Distance/out_metrics_cd.pkl" #MVP 
    balanced_path = "../experiments/Density_aware_Chamfer_Distance/out_metrics_balanced.pkl" #MVP 
else:
    chamfer_path = "sphere/out_metrics_cd.pkl" #PCN
    balanced_path = "sphere/out_metrics.pkl" #PCN

with open (chamfer_path, "rb") as f:
    chamfer_cd, chamfer_emd = pickle.load(f)
    
if dataset == "mvp":
    
    chamfer_cd, chamfer_emd = [x.detach().cpu().numpy() for x in chamfer_cd], [x.detach().cpu().numpy() for x in chamfer_emd]
    chamfer_cd, chamfer_emd = np.array(chamfer_cd).flatten(), np.array(chamfer_emd).flatten()
    chamfer_cd = chamfer_cd*1000
    chamfer_emd = chamfer_emd*100
else:
    chamfer_cd = np.array(chamfer_cd)
    chamfer_emd = np.array(chamfer_emd)*100

with open (balanced_path, "rb") as f:
    balanced_cd, balanced_emd = pickle.load(f)
    
if dataset == "mvp":
    balanced_cd, balanced_emd = [x.detach().cpu().numpy() for x in balanced_cd], [x.detach().cpu().numpy() for x in balanced_emd]
    balanced_cd, balanced_emd = np.array(balanced_cd).flatten(), np.array(balanced_emd).flatten()
    balanced_cd = balanced_cd*1000
    balanced_emd = balanced_emd*100
else:
    balanced_cd = np.array(balanced_cd)
    balanced_emd = np.array(balanced_emd)*100
    
    
print(np.average(balanced_cd), np.average(chamfer_cd), np.average(balanced_emd), np.average(chamfer_emd))


In [ ]:

plt.rcParams['figure.figsize']=(10,6)
plt.xlabel('CD (x$10^4$)', fontsize=20)
plt.ylabel('EMD (x$10^2$)', fontsize=20)
plt.yticks(fontsize=20)
plt.xticks(fontsize=20)
plt.title('MVP Testset Correlation between CD and EMD, VRC model', fontsize=20)

plt.scatter(balanced_cd, balanced_emd, s=3, color="lightseagreen", label="UniformCD loss")
plt.scatter(chamfer_cd, chamfer_emd, s=3, color="palevioletred", label="CD loss")
plt.legend(prop = { "size": 18 })

plt.show()

### find correlation between chamfer and EMD
import numpy as np
import scipy.stats

print("balanced r", scipy.stats.pearsonr(balanced_cd, balanced_emd))
print("chamfer r", scipy.stats.pearsonr(chamfer_cd, chamfer_emd))


#### Visualise loss curves for training VCN models


In [ ]:
# open train log and get eval records
train_log_file = "../experiments/Density_aware_Chamfer_Distance/log/vrcnet_plus_cd_debug_2024-03-11T17:00:03/train - Copy.log"
def get_metrics_from_log(train_log_file, balanced=False):
    with open(train_log_file, "rb") as f:
        train_log = f.readlines()
        
    eval_lines = []
    for line in train_log:
        if "curr" in str(line):
            eval_lines.append(line)
            
            
    # get results
    dcd, cd, emd, bcd = [], [] ,[], []

    for line in eval_lines:
        line = str(line)
        dcd.append(float(line.split("dcd: ")[1].split(";")[0]))
        cd.append(float(line.split("cd_t: ")[1].split(";")[0]))
        emd.append(float(line.split("emd: ")[1].split(";")[0]))
        if balanced:
            bcd.append(float(line.split("bcd_t: ")[1].split(";")[0]))

    if balanced:
        return dcd, cd, emd, bcd
    return dcd, cd, emd


In [ ]:
train_log_file = "../experiments/Density_aware_Chamfer_Distance/log/vrcnet_plus_cd_debug_2024-03-11T17:00:03/train - Copy.log"
dcd_cd, cd_cd, emd_cd  = get_metrics_from_log(train_log_file)

train_log_file = "../experiments/Density_aware_Chamfer_Distance/log/vrcnet_plus_balanced_debug_2024-03-12T23:40:55/train - Copy.log"
dcd_u, cd_u, emd_u, bcd_u = get_metrics_from_log(train_log_file, balanced=True)


In [ ]:
# visualise losses
iterations = list(range(0, len(cd_cd)*2, 2))

plt.rcParams.update({'font.size': 14})
plt.rcParams["figure.figsize"] = (8,6)
plt.rc('xtick', labelsize=16) 
plt.rc('ytick', labelsize=16) 

# Plotting the line graph
#plt.plot(iterations, dcd_cd, label='DCD', color="lightseagreen")

# emd
# plt.plot(iterations, emd_cd, label='CD', color="palevioletred")
# plt.plot(iterations, emd_u, label='UniformCD', color="lightseagreen")
# plt.title('Completion training performance (EMD)')

# cd
# plt.plot(iterations, cd_cd, label='CD', color="palevioletred")
# plt.plot(iterations, cd_u, label='UniformCD', color="lightseagreen")
# plt.title('Completion training performance (CD)')

# dcd
# plt.plot(iterations, dcd_cd, label='CD', color="palevioletred")
# plt.plot(iterations, dcd_u, label='UniformCD', color="lightseagreen")
# plt.title('Completion training performance (DCD)')

# train loss
#plt.plot(iterations, cd_cd, label='CD', color="palevioletred")
plt.plot(iterations, bcd_u, label='UniformCD', color="lightseagreen")
plt.title('Completion training loss')

# Adding labels and title
plt.xlabel('Iterations',  fontsize=16)
plt.ylabel('Loss',  fontsize=16)

# Displaying the graph
plt.legend()
plt.show()



